In [136]:
bff = [2, 3, 4, 1,]
bff = [3, 3, 4, 1,]
bff = [3, 3, 4, 3]
# bff = [7, 8, 10, 10, 9, 2, 9, 6, 3, 3,]
# bff = [int(s, 10) for s in '2 3 1 10 3 5 6 7 8 9'.split()]
bff = [2, 1, 4, 3]
bff = [int(s, 10) for s in '2 1 6 3 8 4 6 5'.split()]

n = len(bff)
c = [[1 if i == bf - 1 else 0 for i in xrange(0, n, 1)] for bf in bff]
adj = [bf - 1 for i, bf in enumerate(bff)]

print 'c = '
print c
print adj


c = 
[[0, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0]]
[1, 0, 5, 2, 7, 3, 5, 4]


In [137]:
# def path(i, adj):
#     print '----'
#     print i
#     p = [i]
#     cur = i
#     n = len(adj)
#     cycle = False
#     while len(p) < n:
#         next = adj[cur]
#         print 'next = %d' % next
#         if next in p:
#             # loop
#             cycle = True
#             break
#         p.append(next)
#         cur = next
#     return p, cycle

class SimplePath(object):
    def __init__(self, i, adj):
        self.nodes = [i]
        self.is_cyclic = False
        self.is_cyclic_tail = False
        self.cycle_start_node_index = None
        
        n = len(adj)
        cur = i
        # print '----'
        for i in xrange(0, n):
            next = adj[cur]
            # print 'next = %d' % next
            self.is_cyclic = next in self.nodes
            if self.is_cyclic:
                # print 'loop'
                # loop
                self.cycle_start_node_index = self.nodes.index(next)
                if next == self.nodes[-2]:
                    self.is_cyclic_tail = True
                break
            self.nodes.append(next)
            cur = next
            
    def __repr__(self):
        return '<SimplePath nodes={nodes}, is_cyclic={is_cyclic}, cycle_start_node_index={cycle_start_node_index}, is_cyclic_tail={is_cyclic_tail}>'.format(
            nodes=self.nodes, is_cyclic=self.is_cyclic, is_cyclic_tail=self.is_cyclic_tail,
            cycle_start_node_index=self.cycle_start_node_index)
    
    def __len__(self):
        return len(self.nodes)
    
    @property
    def cyclic_tail(self):
        return self.nodes[-2:] if self.is_cyclic_tail else None
    
    @property
    def head(self):
        return self.nodes[:-2]
    
    @property
    def is_circle(self):
        if self.is_cyclic == False:
            return False
        else:
            if self.cycle_start_node_index == 0:
                return True
            elif self.is_cyclic_tail == True:
                return True
        return False
    
    @property
    def is_complete_circle(self):
        return self.is_cyclic and self.cycle_start_node_index == 0

    def can_join_path(self, other):
        both_has_cyclic_tail = self.is_cyclic_tail == True and other.is_cyclic_tail == True
        same_tail = self.cyclic_tail[0] == other.cyclic_tail[1] and self.cyclic_tail[1] == other.cyclic_tail[0]
        different_head = set(self.head).isdisjoint(set(other.head))
        return (both_has_cyclic_tail and same_tail and different_head) 
    
    
    
class JoinedPath(object):
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
        
    def __len__(self):
        p1_len = len(self.p1)
        p2_len = len(self.p2)
        if p1_len == 2 or p2_len == 2:
            return p1_len + p2_len
        else:
            return len(self.p1) + len(self.p2) - 2
    
    def __repr__(self):
        return '<JoinedPath path1={p1}, path2={p2}, len={len}'.format(p1=self.p1, p2=self.p2, len=len(self))
    
    
    
def find_answer(n, adj):
    paths = [SimplePath(i, adj) for i in xrange(0, n)]
    valid_paths = [p for p in paths if p.is_circle]
    from pprint import pprint as pp
    #pp(paths)
    complete_circles = []
    lines = []
    for p in valid_paths:
        #pp((p, p.is_circle, len(p)))
        if p.is_complete_circle:
            complete_circles.append(p)
        if len(p) == 2:
            lines.append(p)
        else:
            lines.append(p)

    # TODO: optimize?    
    complete_circles.sort(key=len, reverse=True)    
    pp('circles')
    pp(complete_circles)
    # TODO: 
    lines.sort(key=len, reverse=True)
    pp('lines')
    pp(lines)
    from itertools import combinations
    comb = combinations(lines, 2)

    joinable_line_pairs = [p for p in comb if p[0].can_join_path(p[1])]
    #print 'joinable_line_pairs'
    #pp(joinable_line_pairs)
    joined_paths = [JoinedPath(*pair) for pair in joinable_line_pairs]
    joined_paths.sort(key=len, reverse=True)
    print 'joined_paths'
    pp(joined_paths)

    longest_circle_path_len = len(complete_circles[0]) if complete_circles else -1
    longest_joined_path_len = len(joined_paths[0]) if joined_paths else -1
    longest_lines_len = len(lines[0]) if lines else -1

    return max(longest_circle_path_len, longest_joined_path_len, longest_lines_len)

print find_answer(n, adj)


'circles'
[<SimplePath nodes=[2, 5, 3], is_cyclic=True, cycle_start_node_index=0, is_cyclic_tail=False>,
 <SimplePath nodes=[3, 2, 5], is_cyclic=True, cycle_start_node_index=0, is_cyclic_tail=False>,
 <SimplePath nodes=[5, 3, 2], is_cyclic=True, cycle_start_node_index=0, is_cyclic_tail=False>,
 <SimplePath nodes=[0, 1], is_cyclic=True, cycle_start_node_index=0, is_cyclic_tail=True>,
 <SimplePath nodes=[1, 0], is_cyclic=True, cycle_start_node_index=0, is_cyclic_tail=True>,
 <SimplePath nodes=[4, 7], is_cyclic=True, cycle_start_node_index=0, is_cyclic_tail=True>,
 <SimplePath nodes=[7, 4], is_cyclic=True, cycle_start_node_index=0, is_cyclic_tail=True>]
'lines'
[<SimplePath nodes=[2, 5, 3], is_cyclic=True, cycle_start_node_index=0, is_cyclic_tail=False>,
 <SimplePath nodes=[3, 2, 5], is_cyclic=True, cycle_start_node_index=0, is_cyclic_tail=False>,
 <SimplePath nodes=[5, 3, 2], is_cyclic=True, cycle_start_node_index=0, is_cyclic_tail=False>,
 <SimplePath nodes=[0, 1], is_cyclic=True, cycle

TypeError: 'NoneType' object has no attribute '__getitem__'

In [48]:
def prod(v0, v1, n):
    for i in xrange(0, n):
        if v0[i] != 0 and v1[i] !=0:
            return 1

    return 0
    
def col(m, x):
    return [row[x] for row in m]
    
def mul(m0, m1, n):
    ret = [[0 for i in xrange(0, n)] for j in xrange(0, n)]
    for y in xrange(0, n, 1):
        for x in xrange(0, n):
            ret[y][x] = prod(m0[y], col(m1, x), n)
    return ret

def trace(m, n):
    return sum([m[i][i] for i in xrange(0, n)])
        
def connected(c):
    return [i for i in xrange(0, len(c)) if c[i][i] > 0]

p = 2
c_p = c
cycle = False
max_tr, max_p, max_connected = 0, 0, None
while p <= n :
    c_p = mul(c_p, c, n)
    print 'c^%d = ' % p
    print c_p
    tr = trace(c_p, n)
    print tr
    if tr == n:
        max_tr = tr
        max_p = p
        max_connected = connected(c_p)
        break
    elif tr > max_tr:
        max_tr = tr
        max_p = p
        max_connected = connected(c_p)        
    p += 1
    
print max_tr, max_p, max_connected

c = 
[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]
c^2 = 
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
2
c^3 = 
[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]
3
c^4 = 
[[0, 0,

In [94]:
[1,2,3,4][-3:]


[2, 3, 4]